Nesta etapa, vamos transformar o dataset limpo (`dados_limpos.csv`) em um formato tidy data, seguindo os princípios:
- Cada variável é uma coluna.
- Cada observação é uma linha.
- Cada tipo de unidade observacional forma uma tabela.

Ao final, exportaremos o resultado em formato Parquet, mais eficiente e padronizado.

In [ ]:
import pandas as pd

df = pd.read_csv("dados_limpos.csv")
print("Dimensões iniciais:", df.shape)

## 1)
Padronização dos tipos de dados: converte colunas categóricas em category, numéricas inteiras em Int64 e valores decimais em float com duas casas decimais.

In [ ]:
cols_categoricas = ["TP_SEXO", "SG_UF_PROVA", "Q001", "Q002", "Q006"]
for col in cols_categoricas:
    df[col] = df[col].astype("category")

cols_inteiras = [
    "NU_ANO", "TP_FAIXA_ETARIA", "TP_ESTADO_CIVIL", "TP_COR_RACA",
    "TP_NACIONALIDADE", "TP_ST_CONCLUSAO", "TP_ESCOLA", "IN_TREINEIRO",
    "CO_UF_PROVA", "TP_PRESENCA_CN", "TP_PRESENCA_CH", "TP_PRESENCA_LC",
    "TP_PRESENCA_MT", "TP_LINGUA", "TP_STATUS_REDACAO",
    "NU_NOTA_COMP1", "NU_NOTA_COMP2", "NU_NOTA_COMP3",
    "NU_NOTA_COMP4", "NU_NOTA_COMP5", "NU_NOTA_REDACAO",
    "Q022", "Q024", "Q025", "ANO_REFERENCIA"
]
df[cols_inteiras] = df[cols_inteiras].astype("Int64")

cols_float = [
    "NU_NOTA_CN", "NU_NOTA_CH", "NU_NOTA_LC", "NU_NOTA_MT",
    "PCT_ACERTO_CN", "PCT_ACERTO_CH", "PCT_ACERTO_LC", "PCT_ACERTO_MT"
]
df[cols_float] = df[cols_float].astype(float).round(2)

## 2)
Normalização das variáveis TX_ACERTOS_*, que são sequências de 0 e 1 indicando acertos e erros.
Aqui são criadas colunas numéricas com o total de acertos e o total de questões por área (CN, CH, LC e MT).

In [ ]:
for area in ["CN", "CH", "LC", "MT"]:
    col = f"TX_ACERTOS_{area}"
    if col in df.columns:
        df[f"ACERTOS_{area}"] = df[col].astype(str).apply(lambda x: x.count("1") if pd.notna(x) else None)
        df[f"TOTAL_{area}"] = df[col].astype(str).apply(lambda x: len(x) if pd.notna(x) else None)

## 3)
Transformação do dataset de formato wide para long, reorganizando as notas (NU_NOTA_*) para que cada linha represente um aluno em uma área específica (AREA_CONHECIMENTO).

In [ ]:
id_vars = [
    "NU_ANO", "TP_FAIXA_ETARIA", "TP_SEXO", "TP_ESTADO_CIVIL",
    "TP_COR_RACA", "TP_NACIONALIDADE", "TP_ST_CONCLUSAO", "TP_ESCOLA",
    "IN_TREINEIRO", "CO_UF_PROVA", "SG_UF_PROVA", "TP_LINGUA",
    "TP_STATUS_REDACAO", "NU_NOTA_REDACAO", "ANO_REFERENCIA"
]

df_long_notas = df.melt(
    id_vars=id_vars,
    value_vars=["NU_NOTA_CN", "NU_NOTA_CH", "NU_NOTA_LC", "NU_NOTA_MT"],
    var_name="AREA_CONHECIMENTO",
    value_name="NOTA"
)
df_long_notas["AREA_CONHECIMENTO"] = df_long_notas["AREA_CONHECIMENTO"].str.replace("NU_NOTA_", "")

## 4)
Aplicação da mesma transformação (melt) para as variáveis de percentual de acerto (PCT_ACERTO_*), número de acertos (ACERTOS_*) e total de questões (TOTAL_*).

In [ ]:
df_long_pct = df.melt(
    id_vars=id_vars,
    value_vars=["PCT_ACERTO_CN", "PCT_ACERTO_CH", "PCT_ACERTO_LC", "PCT_ACERTO_MT"],
    var_name="AREA_CONHECIMENTO",
    value_name="PCT_ACERTO"
)
df_long_pct["AREA_CONHECIMENTO"] = df_long_pct["AREA_CONHECIMENTO"].str.replace("PCT_ACERTO_", "")

df_long_acertos = df.melt(
    id_vars=id_vars,
    value_vars=["ACERTOS_CN", "ACERTOS_CH", "ACERTOS_LC", "ACERTOS_MT"],
    var_name="AREA_CONHECIMENTO",
    value_name="ACERTOS"
)
df_long_acertos["AREA_CONHECIMENTO"] = df_long_acertos["AREA_CONHECIMENTO"].str.replace("ACERTOS_", "")

df_long_total = df.melt(
    id_vars=id_vars,
    value_vars=["TOTAL_CN", "TOTAL_CH", "TOTAL_LC", "TOTAL_MT"],
    var_name="AREA_CONHECIMENTO",
    value_name="TOTAL_QUESTOES"
)
df_long_total["AREA_CONHECIMENTO"] = df_long_total["AREA_CONHECIMENTO"].str.replace("TOTAL_", "")

## 5)
Combinação das tabelas geradas na etapa anterior, unindo notas, acertos e percentuais em um único dataset tidy (df_tidy).

In [ ]:
df_tidy = df_long_notas.copy()
df_tidy["PCT_ACERTO"] = df_long_pct["PCT_ACERTO"]
df_tidy["ACERTOS"] = df_long_acertos["ACERTOS"]
df_tidy["TOTAL_QUESTOES"] = df_long_total["TOTAL_QUESTOES"]

print("Dimensões finais (tidy):", df_tidy.shape)
df_tidy.head()

## 6)
Exportação do dataset final em formato Parquet, garantindo compactação, preservação dos tipos de dados e maior eficiência de leitura.


In [ ]:
df_tidy.to_parquet("dados_tidy.parquet", index=False)
print("Dados tidy exportados com sucesso para 'dados_tidy.parquet'")

## 7) Consultas

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
area_labels = {
    "CN": "Ciências da Natureza",
    "CH": "Ciências Humanas",
    "LC": "Linguagens e Códigos",
    "MT": "Matemática"
}

df_tidy["AREA_LABEL"] = df_tidy["AREA_CONHECIMENTO"].map(area_labels)


Análise de tendências temporais: evolução das notas médias por área ao longo dos anos.

In [ ]:
sns.set(style="whitegrid", palette="Set2", font_scale=1.1)

media_ano_area = (
    df_tidy.groupby(["NU_ANO", "AREA_LABEL"])["NOTA"]
    .mean()
    .reset_index()
)

plt.figure(figsize=(10, 6))
sns.lineplot(
    data=media_ano_area,
    x="NU_ANO",
    y="NOTA",
    hue="AREA_LABEL",
    marker="o",
    linewidth=2.5
)

plt.title("Evolução das notas médias por área (ENEM)", fontsize=14, weight="bold")
plt.xlabel("Ano do ENEM")
plt.ylabel("Nota média")
plt.legend(title="Área de Conhecimento", loc="best", frameon=True)
plt.tight_layout()
plt.show()

Comparação entre grupos: médias de nota por sexo e área de conhecimento.

In [ ]:
sexo_labels = {"F": "Feminino", "M": "Masculino"}
media_sexo_area = (
    df_tidy.assign(SEXO=df_tidy["TP_SEXO"].replace(sexo_labels))
    .groupby(["SEXO", "AREA_LABEL"])["NOTA"]
    .mean()
    .reset_index()
)

plt.figure(figsize=(9, 5))
sns.barplot(
    data=media_sexo_area,
    x="AREA_LABEL",
    y="NOTA",
    hue="SEXO",
    palette="pastel",
    edgecolor="gray"
)

plt.title("Comparação de notas médias por sexo e área", fontsize=13, weight="bold")
plt.xlabel("Área de Conhecimento")
plt.ylabel("Nota média")

plt.legend(
    title="Sexo",
    bbox_to_anchor=(1.02, 0.5),
    loc="center left",
    frameon=False
)

plt.xticks(rotation=10)
plt.tight_layout(rect=[0, 0, 0.85, 1]) 
plt.grid(axis="y", linestyle="--", alpha=0.5)
plt.show()

Análise de concentração: distribuição das notas por área.

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(
    data=df_tidy,
    x="AREA_LABEL",
    y="NOTA",
    hue="AREA_LABEL",
    legend=False,
    palette="coolwarm"
)

plt.title("Distribuição das notas por área de conhecimento", fontsize=13, weight="bold")
plt.xlabel("Área de Conhecimento")
plt.ylabel("Nota")
plt.xticks(rotation=15)
plt.tight_layout()
plt.show()

Ranking de desempenho médio por estado (SG_UF_PROVA).

In [ ]:
ranking_estados = (
    df_tidy.groupby("SG_UF_PROVA")["NOTA"]
    .mean()
    .reset_index()
    .sort_values(by="NOTA", ascending=False)
)

ranking_estados["RANK"] = ranking_estados["NOTA"].rank(ascending=False)
ranking_estados.head(27)

Detecção de padrões e anomalias.

In [ ]:
media_area = df_tidy.groupby("AREA_CONHECIMENTO")["NOTA"].mean()
std_area = df_tidy.groupby("AREA_CONHECIMENTO")["NOTA"].std()

df_tidy["Z_SCORE"] = df_tidy.apply(
    lambda x: (x["NOTA"] - media_area[x["AREA_CONHECIMENTO"]]) / std_area[x["AREA_CONHECIMENTO"]],
    axis=1
)

anomalias = df_tidy[df_tidy["Z_SCORE"].abs() > 2]
print(f"Total de possíveis anomalias: {len(anomalias)}")
anomalias.head()

Correlações e dependências entre variáveis.

In [ ]:
corr_df = df_tidy[["NOTA", "PCT_ACERTO", "ACERTOS", "TOTAL_QUESTOES"]].corr()

plt.figure(figsize=(6, 5))
sns.heatmap(
    corr_df,
    annot=True,
    fmt=".2f",
    cmap="Blues",
    square=True,
    cbar_kws={"shrink": 0.8},
    linewidths=0.5
)
plt.title("Correlação entre variáveis principais", fontsize=13, weight="bold")
plt.tight_layout()
plt.show()


Análise hierárquica com CTEs (simuladas via pandas).

In [ ]:
media_estado = df_tidy.groupby("SG_UF_PROVA")["NOTA"].mean().reset_index(name="MEDIA_ESTADO")
media_nacional = media_estado["MEDIA_ESTADO"].mean()

media_estado["DIF_MEDIA_NACIONAL"] = media_estado["MEDIA_ESTADO"] - media_nacional
media_estado.sort_values("MEDIA_ESTADO", ascending=False).head(10)